In [1]:
import pandas as pd
import zipfile
import seaborn as sns

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import gc
import pyarrow.feather as feather

In [3]:
import dask.dataframe as dd

In [4]:
data_path="../data/data_accuracy/"

In [5]:
calender=pd.read_csv(data_path+"calendar.csv",dtype={'event_name_2': 'object',
       'event_type_2': 'object'})
sales_data=pd.read_csv(data_path+"sell_prices.csv")
sales_train_validation=pd.read_csv(data_path+"sales_train_validation.csv")

In [6]:
pd.set_option('display.max_columns', None)

In [7]:
sales_train_validation.shape

(30490, 1919)

In [8]:
s1 = sales_train_validation.groupby(['store_id','dept_id'], group_keys=False).apply(lambda x: x.sample(2))

In [9]:
sales_train_validation = sales_train_validation[sales_train_validation["item_id"].isin(s1["item_id"].unique())]

In [10]:
gc.collect()

141

In [11]:
def snaps(x):
    snaps=[]
    if(x["snap_CA"]==1):
        snaps.append("snapca")
    elif (x["snap_TX"]==1):
        snaps.append("snaptx")
    elif (x["snap_WI"]==1):
        snaps.append("snapwi")
    else:
        snaps.append("snapno")
    return "-".join(snaps)

In [12]:
calender["snaps"] = calender.apply(lambda x: snaps(x),axis=1)

In [13]:
calender["snaps"].value_counts()

snapno    994
snapca    650
snaptx    260
snapwi     65
Name: snaps, dtype: int64

In [14]:
calender.fillna("noevent",inplace=True)

In [15]:
cols = list(sales_train_validation.loc[:,"d_1":].columns)

In [16]:
sales_train_validation.shape

(1350, 1919)

In [17]:
st_validation = pd.melt(sales_train_validation,id_vars=["id","item_id","dept_id","store_id","state_id"],value_vars=cols)

In [18]:
st_validation.head()

,id,item_id,dept_id,store_id,state_id,variable,value
0,HOBBIES_1_033_CA_1_validation,HOBBIES_1_033,HOBBIES_1,CA_1,CA,d_1,0
1,HOBBIES_1_056_CA_1_validation,HOBBIES_1_056,HOBBIES_1,CA_1,CA,d_1,0
2,HOBBIES_1_061_CA_1_validation,HOBBIES_1_061,HOBBIES_1,CA_1,CA,d_1,1
3,HOBBIES_1_063_CA_1_validation,HOBBIES_1_063,HOBBIES_1,CA_1,CA,d_1,0
4,HOBBIES_1_080_CA_1_validation,HOBBIES_1_080,HOBBIES_1,CA_1,CA,d_1,0


In [19]:
st_validation.shape

(2582550, 7)

In [20]:
st_validation = pd.merge(st_validation,calender,left_on="variable",right_on="d")

In [21]:
st_validation.shape

(2582550, 22)

In [22]:
st_validation.columns

Index(['id', 'item_id', 'dept_id', 'store_id', 'state_id', 'variable', 'value',
       'date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year', 'd',
       'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2',
       'snap_CA', 'snap_TX', 'snap_WI', 'snaps'],
      dtype='object')

In [23]:
sales_data.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [25]:
st_validation = pd.merge(st_validation,sales_data,on=["wm_yr_wk","store_id","item_id"],how="left")

In [26]:
st_validation.shape

(2582550, 23)

In [27]:
st_validation.isnull().sum()

id                   0
item_id              0
dept_id              0
store_id             0
state_id             0
variable             0
value                0
date                 0
wm_yr_wk             0
weekday              0
wday                 0
month                0
year                 0
d                    0
event_name_1         0
event_type_1         0
event_name_2         0
event_type_2         0
snap_CA              0
snap_TX              0
snap_WI              0
snaps                0
sell_price      526750
dtype: int64

In [28]:
for col,type1 in zip(st_validation.columns,st_validation.dtypes):
    if(str(type1)=="object"):
        st_validation[col]=st_validation[col].astype("category",copy=False)

In [29]:
feather.write_feather(st_validation,data_path+"agg_sample.feather")